<h1>Patient Survival Prediction</h1>
<hr>
<h2>Project 4</h2>
<p>This project predicts the survival of pacients in a UCI.</p>
<hr>

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

from utilerias import missing_values_table

%matplotlib inline

In [ ]:
df_patients = pd.read_csv('../datasets/Dataset_eda.csv')

<hr>
<h2>Preprocessing</h2>
<hr>
<p>Missing Values Treatment</p>

In [ ]:
df_patients.heads()

In [ ]:
missing_values_table(df_patients).style.background_gradient(cmap='Blues')

In [ ]:
msno.bar(df_patients)

In [ ]:
#lets delete some features
#1) those that seems not related
lstFeaturesToDrop1 = ['encounter_id', 'hospital_admit_source', 'icu_admit_source', 'icu_id', 'icu_stay_type', 'patient_id', 'hospital_id', 'readmission_status']
#2) those with 50% of missing values or more.
lstFeaturesToDrop2 = ['h1_bilirubin_max'
,'h1_bilirubin_min'
,'h1_lactate_min'
,'h1_lactate_max'
,'h1_albumin_max'
,'h1_albumin_min'
,'h1_pao2fio2ratio_min'
,'h1_pao2fio2ratio_max'
,'h1_arterial_ph_min'
,'h1_arterial_ph_max'
,'h1_hco3_min'
,'h1_hco3_max'
,'h1_arterial_pco2_min'
,'h1_arterial_pco2_max'
,'h1_wbc_max'
,'h1_wbc_min'
,'h1_arterial_po2_max'
,'h1_arterial_po2_min'
,'h1_calcium_min'
,'h1_calcium_max'
,'h1_platelets_max'
,'h1_platelets_min'
,'h1_bun_min'
,'h1_bun_max'
,'h1_creatinine_min'
,'h1_creatinine_max'
,'h1_diasbp_invasive_max'
,'h1_diasbp_invasive_min'
,'h1_sysbp_invasive_max'
,'h1_sysbp_invasive_min'
,'h1_mbp_invasive_min'
,'h1_mbp_invasive_max'
,'h1_hematocrit_min'
,'h1_hematocrit_max'
,'h1_hemaglobin_max'
,'h1_hemaglobin_min'
,'h1_sodium_max'
,'h1_sodium_min'
,'h1_potassium_min'
,'h1_potassium_max'
,'ph_apache'
,'pao2_apache'
,'paco2_for_ph_apache'
,'paco2_apache'
,'fio2_apache'
,'d1_lactate_max'
,'d1_lactate_min'
,'d1_diasbp_invasive_min'
,'d1_diasbp_invasive_max'
,'d1_sysbp_invasive_max'
,'d1_sysbp_invasive_min'
,'d1_mbp_invasive_min'
,'d1_mbp_invasive_max'
,'d1_pao2fio2ratio_min'
,'d1_pao2fio2ratio_max'
,'d1_arterial_ph_min'
,'d1_arterial_ph_max'
,'d1_arterial_pco2_max'
,'d1_arterial_pco2_min'
,'d1_arterial_po2_min'
,'d1_arterial_po2_max'
,'bilirubin_apache'
,'h1_inr_max'
,'d1_inr_min'
,'h1_inr_min'
,'d1_inr_max'
,'albumin_apache'
,'d1_bilirubin_min'
,'d1_bilirubin_max'
,'h1_glucose_min'
,'h1_glucose_max'
,'d1_albumin_min'
,'d1_albumin_max'
,'urineoutput_apache']

In [ ]:
df_patients.drop(lstFeaturesToDrop1+lstFeaturesToDrop2,inplace=True)

In [ ]:
#features related each other
df_patients[df_patients[['bmi', 'weight', 'height']].isna().sum(axis=1) == 0]